In [42]:
import os
import pandas as pd

class TickerDataCollector:
    def __init__(self, tickers_file='tickers.csv',results_file='financial_results.csv'):
        """클래스 초기화
        tickers_file : 티커 목록이 저장된 파일
        results_file : 수집된 데이터를 저장할 파일
        """
        self.tickers_file = tickers_file # 티커 목록 파일 경로
        self.results_file = results_file # 결과 파일 경로
        self.tickers_df = pd.DataFrame() # 로드한 티커를 저장할 데이터프레임
        self.results_df = pd.DataFrame() # 수집된 결과를 저장할 데이터프레임

        # 초기 데이터 로드
        self.load_tickers()
        self.load_results()

    def load_tickers(self):
        """티커 목록을 로드하는 메서드"""
        if os.path.exists(self.tickers_file):
            self.tickers_df = pd.read_csv(self.tickers_file)
            # self.tickers = df['Tickers']
            print(f"로드한 티커 : {self.tickers_df['Tickers'][:5]}")
        else:
            print(f"티커 목록 파일 '{self.tickers_file}'을 찾을 수 없습니다. Fetch_Stock_Code을 실행시켜 주세요.")
            sys.exit()

    def load_results(self):
        """이전에 저장된 결과 파일을 로드하는 메서드
        다른 메서드에 비해 우선순위가 낮다. 나중에 개발해도 된다."""
        if os.path.exists(self.results_file):
            self.results_df = pd.read_csv(self.results_file)
        else:
            print(f"결과 파일 '{self.results_file}'을 찾을 수 없습니다.")
            self.results_df = results_df.copy(deep=True) # 결과 데이터프레임에 티커 추가,
            self.results_df.set_index('Tickers', inplace=True) # 티커명을 인덱스로 설정
            self.results_df['Data'] = None # 데이터 열을 추가. 열 제목들은 차후 추가.

    def fetch_data(self,ticker):
        """웹 페이지를 요청하는 메서드.
        파싱한 데이터를 리턴하지만, 파싱 기능은 parse_table에 따로 구현하도록 하자."""
        # 실제 웹 페이지 요청은 나중에 구현 (requests.get()사용)
        print(f"데이터를 수집하는 티커 : {ticker}")

    def parse_table(self, html_data):
        """데이터를 파싱하는 메서드. 파싱하기 위한 html 태그들은 이미 확보하였음."""
        # 실제 데이터 파싱 로직은 나중에 구현(BeautifulSoup 사용)
        print(f"파싱 중 : {html_data}")
        return ["Parsed Data"] # 더미 파싱 결과 (실제 파싱 로직으로 대체)

    def collect_data(self):
        """모든 티커에 대해 데이터를 수집하는 메서드.
        for문을 이용해 기존에 탐색했던 구간부터, 문서의 끝까지 탐색해야 한다."""
        for ticker in self.results_df.index[:10]:
            html_data = self.fetch_data(ticker)
            parsed_data = self.parse_table(html_data)
            self.results_df.loc[ticker] = parsed_data
            if len(self.results_df) % 10 == 0:
                self.save_results()
    def save_results(self):
        """수집된 데이터를 CSV 파일로 저장하는 메서드.
        메모리 과부화를 막기 위해 10개의 티커마다 저장하도록 기능하면 좋겠다."""
        print(f"결과를 저장하는 중 : {self.results_file}")
        self.results_df.to_csv(self.results_file, index=False)

if __name__ == "__main__":
    tickers_file = 'tickers.csv' # 티커 파일 경로
    financial_results_file = 'financial_results.csv' # 결과 파일 경로

    collector = TickerDataCollector(tickers_file, financial_results_file) # 객체 생성
    collector.collect_data() # 데이터 수집 시작


로드한 티커 : 0    CALC
1    MPTI
2    APLE
3     VLO
4     PBA
Name: Tickers, dtype: object
데이터를 수집하는 티커 : 0
파싱 중 : None
결과를 저장하는 중 : financial_results.csv
데이터를 수집하는 티커 : 1
파싱 중 : None
결과를 저장하는 중 : financial_results.csv
데이터를 수집하는 티커 : 2
파싱 중 : None
결과를 저장하는 중 : financial_results.csv
데이터를 수집하는 티커 : 3
파싱 중 : None
결과를 저장하는 중 : financial_results.csv
데이터를 수집하는 티커 : 4
파싱 중 : None
결과를 저장하는 중 : financial_results.csv
데이터를 수집하는 티커 : 5
파싱 중 : None
결과를 저장하는 중 : financial_results.csv
데이터를 수집하는 티커 : 6
파싱 중 : None
결과를 저장하는 중 : financial_results.csv
데이터를 수집하는 티커 : 7
파싱 중 : None
결과를 저장하는 중 : financial_results.csv
데이터를 수집하는 티커 : 8
파싱 중 : None
결과를 저장하는 중 : financial_results.csv
데이터를 수집하는 티커 : 9
파싱 중 : None
결과를 저장하는 중 : financial_results.csv
